In [30]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time 
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query

import random
import time
import datetime

import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

from procurement_lib import GoogleSheet

In [31]:
todays_date = datetime.datetime.today().strftime('%Y-%m-%d')

In [32]:
query = """
select
    competitor.competitor_name,
    site.identifier_value as site_code,
    quotation_date.full_date AS quotation_date,
    su.source_id,
    ROUND(AVG(cpp.product_selected_price),2)::float as price
from dpr_product_pricing.fact_collected_product_prices cpp
    inner join dpr_shared.dim_date quotation_date
        on cpp.dim_quotation_date = quotation_date.date_id
    inner join dpr_shared.dim_time quotation_time
        on cpp.dim_quotation_time = quotation_time.time_id
    inner join dpr_shared.dim_site site
        on cpp.dim_site = site.site_id
    inner join dpr_shared.dim_category cat
        on cpp.dim_category = cat.category_id
    inner join dpr_product_pricing.dim_product_outlier_type outlier_type
        on cpp.dim_outlier_type = outlier_type.outlier_type_id
    inner join dpr_product_pricing.dim_product_source_type source_type
        on cpp.dim_source_type = source_type.source_type_id
    inner join dpr_product_pricing.dim_product_competitor competitor
        on cpp.dim_competitor = competitor.competitor_id
    inner join dpr_product_pricing.dim_product_competitor_type competitor_type
        on(
            case
                when cpp.super_category = 'Fruver'
                    then competitor.product_competitor_type_id_fruver = competitor_type.competitor_type_id
                when cpp.super_category = 'Multicategoría'
                    then competitor.product_competitor_type_id_multicategoria = competitor_type.competitor_type_id
            end
        )
    inner join dpr_shared.dim_stock_unit su
        on cpp.dim_stock_unit = su.stock_unit_id
where quotation_date.full_date >= current_date - 10
    AND competitor.competitor_name NOT ILIKE '%cayena%'
    AND site.identifier_value IN ('SPO','CWB','VCP','BHZ')
GROUP BY 1,2,3,4
"""
df_zkkkkk = run_read_dwd_query(query)

df_zkkkkk = df_zkkkkk.dropna().reset_index(drop=True)
df_zkkkkk['lifetime'] = 8

In [33]:
# Ensure dataframe is sorted by 'quotation_date'
df_zkkkkk = df_zkkkkk.sort_values(by='quotation_date')

# Generate the required rows for missing dates
new_rows = []

for (competitor, source_id), group in df_zkkkkk.groupby(['competitor_name', 'source_id']):
    group = group.sort_values(by='quotation_date')
    last_known_price = None
    last_known_date = None
    lifetime = 8
    
    for current_index in range(len(group)):
        current_date = group.iloc[current_index]['quotation_date']
        price = group.iloc[current_index]['price']
        
        # If this is not the first iteration, fill in missing dates
        if last_known_date is not None:
            days_diff = (current_date - last_known_date).days
            if days_diff > 1:
                for j in range(1, min(days_diff, lifetime + 1)):
                    new_date = last_known_date + timedelta(days=j)
                    new_row = {
                        'site_code': group.iloc[current_index]['site_code'],
                        'quotation_date': new_date,
                        'competitor_name': competitor,
                        'source_id': source_id,
                        'price': last_known_price,
                        'lifetime': lifetime - j
                    }
                    new_rows.append(new_row)
                    
                    # Stop if we reach a new datapoint date
                    if new_date + timedelta(days=1) == current_date:
                        break
        
        # Update the last known values and reset lifetime
        last_known_price = price
        last_known_date = current_date
        lifetime = 8  # Reset lifetime

    # After processing all known dates for the group, continue generating rows until lifetime reaches 0
    while lifetime > 0:
        last_known_date += timedelta(days=1)
        new_row = {
            'site_code': group.iloc[-1]['site_code'],
            'quotation_date': last_known_date,
            'competitor_name': competitor,
            'source_id': source_id,
            'price': last_known_price,
            'lifetime': lifetime - 1
        }
        new_rows.append(new_row)
        lifetime -= 1

# Append new rows to the dataframe
df_zkkkkk = df_zkkkkk.append(new_rows, ignore_index=True)

# Sort the final dataframe
df_zkkkkk = df_zkkkkk.sort_values(by=['competitor_name', 'source_id', 'quotation_date'])
df_zkkkkk['replica'] = df_zkkkkk['lifetime'] == 8

In [34]:
df_bench = df_zkkkkk.copy()
df_bench['quotation_date'] = pd.to_datetime(df_bench['quotation_date'])

In [35]:
df_c = df_bench.loc[df_bench.quotation_date == todays_date].groupby("source_id").agg(competitors=('competitor_name',lambda x: x.nunique())).reset_index()

In [36]:
query = """
with all_prices as(
    select
        date.full_date as fecha,
        site.identifier_value as region,
        sup.source_id as product_ids,
        sup.description as product_name,
        min_base_price as price,
        min_sale_price as sale_price,
        min_pricing_price as price_hook_acmktpl,
        -- pp.product_price_id,
        row_number() over(partition by sup.source_id,date_created_at_id,time_created_at_id order by su.min_weight_unit desc, price asc) as family_order
    from dpr_product_pricing.dim_product_current_price pp
        inner join dpr_shared.dim_stock_unit su
            on pp.stock_unit_id = su.stock_unit_id
        inner join dpr_shared.dim_stock_unit sup
            on nvl(nullif(su.source_parent_id,0),su.source_id) = sup.source_id
        inner join dpr_shared.dim_date date
            on pp.date_created_at_id = date.date_id
        inner join dpr_shared.dim_site site
            on su.site_id = site.site_id
        inner join dpr_shared.dim_category cat
            on sup.category_id = cat.category_id
    where date.full_date >= current_date - 60
        and date.full_date <= current_date
        and su.description not like '%(S - I)%'
        and cat.super_category = 'Multicategoría'
        and su.active = 1
        and su.description not like '%( S - I)%'
        and su.description not like '%FDL%'
        and su.description not like '%Robin-Food%'
        and su.description not like '%Robin Food%'
        and su.description not like '%RF%'
        and su.description not like '%PPY%'
        and su.description not like '%KAM%'
        and su.description not like '%SIN IVA%'
        and su.description not like '%Oakberry%'
),

clusters as (
 SELECT 
        batch_id,
        created_at,
        last_modified_at,
        sku,
        stock_unit_id,
        source_id,
        type AS cluster,
        site_id,
        ROW_NUMBER() OVER (PARTITION BY sku ORDER BY last_modified_at DESC, cluster DESC) AS rn
    FROM 
        dpr_product_pricing.dim_sku_cluster_period
    where 
        site_id in (4,6,9,11)
)


Select
    fecha.full_date as quotation_date,
    all_prices.region,
    su.source_id,
    --su.description as product_name,
    clusters.cluster,
    price::FLOAT AS price,
    sale_price::FLOAT AS sale_price,
    price_hook_acmktpl::FLOAT AS precio_neto,
    (CASE WHEN c.dtd_cost_local = 0 THEN m.inventory_p_fin ELSE c.dtd_cost_local END)::FLOAT  AS costo,
    min_price::FLOAT AS bench
    
from all_prices
    inner join dpr_shared.dim_stock_unit su 
        on all_prices.product_ids = su.source_id
        
    left join dpr_cross_business.int_dtd_cost c 
        on c.dim_stock_unit = su.stock_unit_id
    
    inner join dpr_shared.dim_date fecha_id 
        on c.dim_date_dtd = fecha_id.date_id
    inner join dpr_shared.dim_date fecha 
        on fecha_id.full_date+1 = fecha.full_date

    left join dpr_cross_business.fact_cross_business_insights m 
        on m.dim_stock_unit = su.stock_unit_id AND m.dim_date = fecha.date_id
    
    left join clusters on clusters.sku = su.sku
    
    left join dpr_product_pricing.obt_benchmark_product_prices b
        on b.stock_unit_id = su.stock_unit_id AND b.benchmark_date = fecha.full_date

where current_date = fecha.full_date
and family_order = 1
and rn = 1
"""
df = run_read_dwd_query(query)
df['quotation_date'] = pd.to_datetime(df['quotation_date'])

In [37]:
df = pd.merge(df, df_c, left_on=['source_id'], right_on=['source_id'], how='left')

In [38]:
df.head()

,quotation_date,region,source_id,cluster,price,sale_price,precio_neto,costo,bench,competitors
0,2024-06-17,SPO,633105,TAIL,52.04,52.04,52.04,32.0625,NaN,NaN
1,2024-06-17,VCP,632885,TAIL,41.88,41.88,41.88,33.9600,NaN,NaN
2,2024-06-17,VCP,476889,MID,15.35,15.35,15.35,12.6890,14.30,1.0
3,2024-06-17,SPO,73465,KVI,4.89,4.89,4.89,4.1027,4.89,17.0
4,2024-06-17,BHZ,641000,KVI,12.28,12.28,12.28,10.4908,NaN,NaN


In [39]:
# Función para calcular el percentil
def calcular_percentil(row, bench_df):
    source_id = row['source_id']
    quotation_date = row['quotation_date']
    precio = row['precio_neto']
    
    # Filtrar precios de referencia por source_id y quotation_date
    bench_precios = bench_df[(bench_df['source_id'] == source_id) & (bench_df['quotation_date'] == quotation_date)]['price']
    
    # Si no hay datos de referencia para ese source_id y quotation_date, devolver NaN
    if bench_precios.empty:
        return np.nan
    
    # Convertir a un array de numpy
    bench_precios = bench_precios.to_numpy(dtype=np.float64)
    
    # Manejo de valores por encima del máximo y por debajo del mínimo
    if precio < np.min(bench_precios):
        return -100*(1-(precio/np.min(bench_precios))) # Percentil mínimo
    elif precio > np.max(bench_precios):
        return 100*(precio/np.max(bench_precios)) # Percentil maximo
    
    # Calcular el percentil
    percentil = stats.percentileofscore(bench_precios, precio)
    return percentil

# Aplicar la función a cada fila de df
df['percentil'] = df.apply(calcular_percentil, axis=1, bench_df=df_bench)

In [40]:
df.head()

,quotation_date,region,source_id,cluster,price,sale_price,precio_neto,costo,bench,competitors,percentil
0,2024-06-17,SPO,633105,TAIL,52.04,52.04,52.04,32.0625,NaN,NaN,NaN
1,2024-06-17,VCP,632885,TAIL,41.88,41.88,41.88,33.9600,NaN,NaN,NaN
2,2024-06-17,VCP,476889,MID,15.35,15.35,15.35,12.6890,14.30,1.0,107.342657
3,2024-06-17,SPO,73465,KVI,4.89,4.89,4.89,4.1027,4.89,17.0,8.823529
4,2024-06-17,BHZ,641000,KVI,12.28,12.28,12.28,10.4908,NaN,NaN,NaN


In [41]:
def calcular_precio_por_percentil_row(row, bench_df, desired_percentil):
    source_id = row['source_id']
    quotation_date = row['quotation_date']
    
    # Filtrar precios de referencia por source_id y quotation_date
    bench_precios = bench_df[(bench_df['source_id'] == source_id) & (bench_df['quotation_date'] == quotation_date)]['price']
    
    # Si no hay datos de referencia para ese source_id y quotation_date, devolver NaN
    if bench_precios.empty:
        return np.nan
    
    # Convertir a un array de numpy y ordenar los precios
    bench_precios = bench_precios.to_numpy(dtype=np.float64)
    bench_precios.sort()
    
    # Calcular el precio correspondiente al percentil deseado
    precio = np.percentile(bench_precios, desired_percentil)
    
    return precio

In [42]:
# Desired percentile (e.g., 25th percentile)
desired_percentil = 25

# Apply the function to each row in df
df['p25'] = df.apply(lambda row: calcular_precio_por_percentil_row(row, df_bench, desired_percentil), axis=1)

In [43]:
df.head()

,quotation_date,region,source_id,cluster,price,sale_price,precio_neto,costo,bench,competitors,percentil,p25
0,2024-06-17,SPO,633105,TAIL,52.04,52.04,52.04,32.0625,NaN,NaN,NaN,NaN
1,2024-06-17,VCP,632885,TAIL,41.88,41.88,41.88,33.9600,NaN,NaN,NaN,NaN
2,2024-06-17,VCP,476889,MID,15.35,15.35,15.35,12.6890,14.30,1.0,107.342657,14.30
3,2024-06-17,SPO,73465,KVI,4.89,4.89,4.89,4.1027,4.89,17.0,8.823529,4.99
4,2024-06-17,BHZ,641000,KVI,12.28,12.28,12.28,10.4908,NaN,NaN,NaN,NaN


In [44]:
df['mg'] = (1-df.costo/df.precio_neto)
df['mg_p25'] = (1-df.costo/df.p25)

In [45]:
query = """
SELECT
    s.identifier_value AS city,
    cat.parent_description AS cat,
    sup.source_id,
    s.identifier_value || ' || ' || sup.card_description AS product_name,
    (SUM(fs.gmv_pxq_local)/4.75)::FLOAT AS gmv_usd
FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
INNER JOIN dpr_shared.dim_stock_unit        su  ON su.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_stock_unit        sup  ON nvl(nullif(su.source_parent_id,0),su.source_id) = sup.source_id

WHERE 
    fs.gmv_enabled = TRUE
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND cat.super_category = 'Multicategoría'
    AND fs.dim_status = 1
    AND dp.is_slot = 'false'
    AND fs.gmv_pxq_local > 0
    AND s.identifier_value IN ('SPO','CWB','VCP','BHZ')
    AND DATE(fs.order_submitted_date) >= CURRENT_DATE - 15
GROUP BY 1,2,3,4
"""
df_gmv = run_read_dwd_query(query)

In [46]:
merge_df = pd.merge(df_gmv, df, left_on=['source_id'], right_on=['source_id'], how='left')

merge_df['gmv_mix'] = merge_df['gmv_usd'] / merge_df.groupby('city')['gmv_usd'].transform('sum')

merge_df = merge_df[['city', 'quotation_date', 'cat', 'source_id', 'cluster', 'product_name', 'competitors','gmv_usd', 'gmv_mix',
       'price', 'sale_price', 'precio_neto', 'bench', 'costo', 'percentil', 'p25', 'mg', 'mg_p25', ]]

In [47]:
merge_df['uplift'] = 100.00*((merge_df.p25/merge_df.precio_neto)-1) #es cuanto deberia subir mi precion en %
merge_df['npi'] = merge_df.precio_neto/merge_df.bench
merge_df['npi_p'] = merge_df.p25/merge_df.bench

In [48]:
# Tiene que tener al menos 5 competidores y estar abajo del percentil 25 !!
merge_df.loc[(merge_df.competitors >= 5) & (merge_df.percentil <= 25)].groupby("city").gmv_mix.sum()

city
BHZ    0.364609
CWB    0.265797
SPO    0.400690
VCP    0.271372
Name: gmv_mix, dtype: float64

In [49]:
df_x = merge_df.loc[(merge_df.competitors >= 5) & (merge_df.percentil <= 25)].copy()

In [50]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [51]:
# Create the histogram
fig = px.histogram(df_x, x='uplift', nbins=20, marginal='box', 
                   title='Distribution of Uplift',
                   labels={'uplift': 'Uplift'},
                   color_discrete_sequence=['blue'])

# Update layout for better aesthetics
fig.update_layout(
    xaxis_title='Uplift',
    yaxis_title='Frequency',
    title_font_size=20,
    xaxis_title_font_size=16,
    yaxis_title_font_size=16,
    bargap=0.3
)

# Show the plot
fig.show()

In [52]:
# df_x.to_excel("info_pipe_filter.xlsx",index=False)

In [53]:
dfg = pd.read_csv("info_gaby.csv")

In [54]:
df_x = pd.merge(df_x, dfg, left_on=['source_id'], right_on=['source_id'], how='left')

In [55]:
df_x.head()

,city,quotation_date,cat,source_id,cluster,product_name,competitors,gmv_usd,gmv_mix,price,...,costo,percentil,p25,mg,mg_p25,uplift,npi,npi_p,mean_edpv,elasticity
0,SPO,2024-06-17,Laticínios e ovos,606541,MID,SPO || Requeijão Cremoso 1.5kg - Tirolez,12.0,16057.8863,0.003574,43.31,...,38.9840,25.000000,45.37,0.099885,0.140754,4.756407,1.031682,1.080753,0.065041,sup. elastic
1,SPO,2024-06-17,Bebidas,277873,KVI,SPO || Coca-Cola 350ml,17.0,64871.4084,0.014437,3.29,...,2.5232,17.647059,3.45,0.233070,0.268638,4.863222,0.953623,1.000000,0.028826,elastic
2,SPO,2024-06-17,Bebidas,614937,MID,SPO || Energético Red Bull 250ml,16.0,13156.9347,0.002928,7.89,...,6.6399,18.750000,7.99,0.158441,0.168974,1.267427,1.012837,1.025674,0.098756,sup. elastic
3,SPO,2024-06-17,Mercearia,632839,TAIL,SPO || Açúcar Cristal Caravelas 1kg,9.0,7052.5536,0.001570,3.55,...,3.2000,11.111111,3.59,0.098592,0.108635,1.126761,0.988858,1.000000,0.104263,sup. elastic
4,SPO,2024-06-17,Congelados,243663,KVI,SPO || Batata Pré-Frita Corte Tradicional 2.5k...,17.0,15743.8126,0.003504,33.99,...,31.1610,23.529412,34.88,0.083230,0.106623,2.618417,1.010855,1.037323,0.046539,elastic


In [56]:
# Create a boxplot
fig = px.box(df_x, y='uplift', title='uplift Boxplot', labels={'uplift': 'uplift Amount'})

# Show the plot
fig.show()

In [62]:
df_x.groupby('city').gmv_mix.sum()

city
BHZ    0.364609
CWB    0.265797
SPO    0.400690
VCP    0.271372
Name: gmv_mix, dtype: float64

In [57]:
# Create a boxplot
fig = px.box(df_x, y='mean_edpv', title='mean_edpv Boxplot', labels={'mean_edpv': 'mean_edpv Amount'})

# Show the plot
fig.show()

## FALTA AÑADIR LAS 2 COLUMNAS DEL DE GABY

# Falta conectar:

hay que hacer match y calcular principalmente:
-> percentiles diario de la dispersion del uplift (como un boxplot)
-> percentilues diario de la elasticidad gaby (como un boxplot)


i.e. El ejemplo de union es a la inversa, es decir imaginemos que el oleo tiene un uplift que esta en el P75 y su elasticidad esta en el P30 que representa 


nuevos campos:
- new price
- new percentil
- new npi
- new margin
- new uplift